In [1]:
import os
import time
from dotenv import load_dotenv
import textwrap
from pydantic import BaseModel, Field

from langchain.chains import GraphCypherQAChain

from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import PDFPlumberLoader

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts.prompt import PromptTemplate

from langchain_experimental.text_splitter import SemanticChunker
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Warning control
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

True

# Connect to Neo4J Database

In [2]:
# Connect to local Neo4J database
NEO4J_URI = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [3]:
# Clear database
graph.query("""
MATCH (n)
DETACH DELETE n
""")
try:
    graph.query("""DROP INDEX doc_chunks""")
except:
    print("No such index")

No such index


In [4]:
# loader = PDFPlumberLoader("data/ISBANK2023.pdf")
# docs = loader.load()
# text_splitter = SemanticChunker(HuggingFaceEmbeddings())
# documents = text_splitter.split_documents(docs)
# # Check the number of pages
# print("Number of pages in the PDF:",len(docs))
# print("Number of documents after chunking:",len(documents))

# # # Merge multiple documents into a single input for unified graph generation
# # combined_content = "\n\n".join([doc.page_content for doc in documents[50:100]]) 
# # combined_content = [Document(page_content=combined_content)]


In [5]:
# for doc in documents:
#     if "\n" in doc.page_content:
#         print(doc)
#         doc.page_content = doc.page_content.replace("\n", "")

# for i, doc in enumerate(documents):
#     if len(doc.page_content) <= 10:
#         documents.pop(i)

# print("Number of pages in the PDF:",len(docs))
# print("Number of documents after chunking:",len(documents))

In [40]:
fake_text = """
Umut Can Gulsen, born in 1998, was a Turkish and naturalised-French physicist and chemist who conducted pioneering research on quantum physics.
He was the first man to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. His husband, Pierre Gulsen, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
He was, in 200, the first man to become a professor at the University of Paris. 
"""

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
isbank_text = """
İşbank, officially Türkiye İş Bankası,[a] is a commercial bank in Turkey. Founded by the orders of Mustafa Kemal Atatürk in 1924, it is the first bank to go into operation in the Republic of Türkiye.
As of a late 2023 report, the bank operates with 1.066 branches domestically, giving it one of the largest branch networks in Turkey. İşbank also has 22 branches across 11 different countries. The bank provides services to 24.3 million customers as of 2023.[6]

History
Following the culmination of World War I in 1918 and the subsequent dissolution of the Ottoman Empire in 1922, Turkey was declared a republic with Mustafa Kemal Atatürk as the president. Atatürk acknowledged the government’s need for a national financial institution to rebuild Turkey's economy following the debacle of the war. İşbank was founded on 26 August 1924 at the First Economy Congress in İzmir.[7]
Atatürk appointed Celâl Bayar, his close aide and then the Minister of Exchange Construction and Settlement, as the president of the newly formed bank. İşbank began operations with two branches and 37 staff under the leadership of Celâl Bayar, its first general manager. The bank was established with a capital of 1,000,000 TL of which 250,000 TL was covered by Atatürk himself, and the rest by private investors.[8] In 1927, the capital was raised by 2 Million TL so it could be merged with the National Credit Bank as equal partners.[9]
Established to utilize the smallest savings and transfer them to development, İşbank played a major role in the establishment and development of savings awareness in Turkish society, bringing the first piggy banks to Turkey in 1928.[10] In the following years, İşbank began to expand by opening branches across the country. In 1932, İşbank opened branches in Hamburg, Germany and Alexandria, Egypt, becoming the first Turkish bank to open branches abroad. In the 1950s,[11] İşbank expanded its portfolio of subsidiaries. The bank's subsidiaries became the driving force of Turkish industry, investing in and financing a wide range of businesses, particularly in the manufacturing sector. Having accelerated branch expansion in the 1960s and 1970s, İşbank paid special attention to the development of its overseas branch network in the 1980s. In the 1980s, İşbank began to offer a wide range of services to its customers in line with its multichannel banking vision. In 1982, İşbank introduced the first Automated Teller Machines (ATM) to Turkey, and at the same time, the bank named its ATMs as "Bankamatik" and registered this name as a trademark. With the launch of the "Blue Line" in 1991[12] and the first internet branch in 1997, İşbank assumed a pioneering role in this field. Since those years, İşbank has been carrying out R&D activities and studies on the use of new technologies to increase the quality and diversity of its services in line with the wishes and expectations of its customers.
"""

scifi_text = """
The Chronomancer’s Emissary
Prologue

In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy, allowing humanity to traverse time and alter its course. However, meddling with time had unforeseen consequences. The universe was fracturing, time was leaking, and events from other periods bled unpredictably into the present. To counteract the chaos, the Chronomancers established the Temporal Enforcement Agency, or TEA, which sent specialized agents known as Emissaries to repair the timelines and prevent collapse.

One such Emissary was Kaden Sato. Strong-willed and fiercely intelligent, Kaden had been chosen for his unparalleled understanding of quantum mechanics and his bravery in the face of uncharted danger. Little did he know that he would soon embark on a mission unlike any before, one that would change the fate of time itself.

Chapter 1: The Ripple

Kaden sat in the briefing room of the TEA headquarters, his face illuminated by the cold blue light of the holographic interface in front of him. The Chronomancer Council, an assembly of the most brilliant minds in temporal science, looked down upon him through the hologram, their faces obscured by shadows.

“Kaden,” began Dr. Myra Elara, a stoic yet compassionate woman who had been Kaden’s mentor for years. “We’ve detected an anomaly in the year 2024. It’s… different from previous incidents. It’s not just an object or a person out of place. It’s an entire event.”

“An event?” Kaden raised an eyebrow, intrigued.

“Precisely,” she replied. “It’s something we call a Ripple Event. Somehow, a monumental event from 2145 has slipped back in time and is altering the entire flow of history in 2024.”

“What’s the event?” Kaden asked, leaning forward.

“It’s the Fusion Accord – the day humanity achieved unlimited energy. If this event occurs prematurely, the energy infrastructure of 2024 won’t handle it, potentially leading to a cataclysmic overload and collapse of global civilization.”

Kaden’s heart pounded. He’d been trained for irregularities, but an entire event displacement was unprecedented.

“You’ll be sent back to 2024 to correct this,” Myra continued, “and you’ll be given access to advanced temporal suppression equipment. Your mission is to locate the epicenter of the Ripple, determine the cause, and neutralize it. Time is already destabilizing; the longer we wait, the greater the damage.”

Kaden nodded, feeling the weight of the task.

Chapter 2: Arrival in 2024

The temporal displacement was jarring, as it always was. Kaden found himself standing in a busy street in New York City, his body adjusting to the unfamiliar sounds, smells, and sights of an era that felt almost quaint to him. People brushed past him, unaware of the massive temporal energy fluctuation he had just unleashed upon arrival.

He took a deep breath, activating his Temporal Inhibitor, a device strapped to his wrist that helped conceal his presence from anyone actively looking for disruptions in the timeline. The Inhibitor also fed him real-time data about the surrounding area. Glancing down, he saw an alert flashing: Anomaly detected. Epicenter identified - East Village Laboratory, January 12, 2024.

Making his way to the lab, Kaden felt an unshakable sense of urgency. As he approached, he noticed a heavy military presence, far too significant for a standard laboratory. Were they aware of the anomaly?

He quickly activated his Temporal Cloak, allowing him to move unseen, and slipped into the facility. He found the lab filled with computers, scientists, and a device he instantly recognized – a rudimentary fusion reactor, decades ahead of its time.

“Impossible…” he whispered, approaching the reactor. His Inhibitor pinged with information: Temporal Signature: Year 2145.

Someone had brought the technology from the future. But who?

Chapter 3: The Conspiracy

Kaden’s investigation led him to a series of encrypted files on one of the lab computers. Decoding them, he discovered the existence of a clandestine organization known as “The Continuum,” a rogue faction of scientists and engineers who believed the TEA was restricting humanity’s potential by controlling the timeline.

The Continuum’s manifesto declared that “Time belongs to all of humanity, not just the chosen few,” and they had been manipulating events throughout history to accelerate technological advancement. It was their belief that humanity was ready for fusion power, regardless of the instability it would cause.

Kaden realized the stakes were higher than he had initially thought. This wasn’t just a single incident; it was part of a larger plan to rewrite human history.

Chapter 4: The Shadowed Ally

Kaden soon discovered that he was not the only Emissary in 2024. Rumors whispered of another agent who had gone rogue, possibly working with the Continuum. Her name was Elise Tanaka, and she was once the finest operative TEA had ever trained. She had disappeared years ago, presumed dead in an accident involving temporal energy overload.

Tracking her movements across 2024, Kaden found her in an abandoned building in Brooklyn. She looked at him with a wry smile, her face illuminated by the soft glow of a fusion core.

“So they finally sent you,” she said, her voice a strange mixture of bitterness and relief.

“Elise,” he whispered, “why did you join them?”

“They’re not wrong, Kaden,” she replied, her eyes flashing with conviction. “Time shouldn’t be controlled. It should be free, accessible to everyone. The TEA isn’t protecting history; they’re holding humanity back.”

“By destabilizing entire centuries? You’re playing with forces you barely understand.”

“Maybe,” she admitted. “But we’ve grown too cautious, too afraid of change.”

He realized she was not the villain he had imagined. She truly believed that by unleashing technological advances, she was giving humanity a chance to evolve.

But he couldn’t let her continue.

Chapter 5: The Showdown

The climax of their struggle took place in the heart of the Continuum’s secret headquarters – a hidden facility beneath the streets of Manhattan, where the stolen fusion reactor was kept. As Kaden fought his way through Elise’s allies, he saw visions of the future flickering around him – alternate timelines where humanity had risen to great heights and others where it had been reduced to ashes.

At the core of the facility, Kaden and Elise stood face to face, each determined to fight for their beliefs.

“You’re afraid of progress,” she accused him.

“No,” he replied. “I’m afraid of chaos.”

Their battle was fierce, with temporal energy crackling around them. Every strike from their temporal weapons rippled across the room, creating fractures in reality. Moments from different eras appeared, and then dissolved: ancient Rome, World War II, a distant Martian colony.

Finally, Kaden managed to disable Elise’s Temporal Inhibitor, leaving her vulnerable.

“It doesn’t have to end like this,” he pleaded. “Come back with me. We can find a way to use this technology responsibly.”

Elise’s face softened, but she shook her head. “You don’t understand. This isn’t just a mission for me. It’s my life’s work.”

Before he could respond, she activated the fusion reactor. It surged with energy, dangerously close to causing a catastrophic time rift.

Kaden knew he had one choice: he activated his Inhibitor’s failsafe, sending an overload pulse to disrupt the fusion core. It would send both him and Elise back to their original timeline, but would hopefully stop the rift.

As the energy surrounded them, he saw Elise’s face one last time, her expression unreadable.

Epilogue

Kaden awoke in the infirmary of TEA headquarters, his mind reeling from the mission. He had succeeded in preventing the catastrophic fusion event, but at a great cost. Elise was gone, and the Continuum had fractured.

In the days that followed, he often thought of her – of her passion and conviction, and of the blurred line between hero and villain in a world where time itself was a battlefield. He knew the TEA would continue its work, but he also knew that Elise’s vision of a free timeline would always haunt him.

As he stared out into the sprawling city beyond TEA headquarters, he wondered if one day humanity would be ready to take charge of its own destiny… and if he would live to see it.
"""
# Convert the text into documents
documents = [Document(page_content=scifi_text)]

In [41]:
for doc in documents:
    if "\n" in doc.page_content:
        doc.page_content = doc.page_content.replace("\n\n", "\n")

In [42]:
documents

[Document(metadata={}, page_content='\nThe Chronomancer’s Emissary\nPrologue\nIn the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy, allowing humanity to traverse time and alter its course. However, meddling with time had unforeseen consequences. The universe was fracturing, time was leaking, and events from other periods bled unpredictably into the present. To counteract the chaos, the Chronomancers established the Temporal Enforcement Agency, or TEA, which sent specialized agents known as Emissaries to repair the timelines and prevent collapse.\nOne such Emissary was Kaden Sato. Strong-willed and fiercely intelligent, Kaden had been chosen for his unparalleled understanding of quantum mechanics and his bravery in the face of uncharted danger. Little did he know that he would soon embark on a mission unlike any before, one that would change t

In [43]:
# Initialize the language model for text-to-graph conversion
llm = ChatOllama(model="llama3", temperature=0)
llm_transformer_filtered = LLMGraphTransformer(llm=llm)

In [44]:
# # Convert multiple text documents into graph structures
# graph_documents = []
# for doc in docs[:10]:  # Iterate over multiple documents
#     graph_documents.extend(llm_transformer_filtered.convert_to_graph_documents([doc]))


In [48]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

In [55]:
split_text = text_splitter.split_text(documents[0].page_content)

In [60]:
split_documents = []
for chunk in split_text:
    split_documents.append(Document(page_content=chunk))

In [ ]:
# Convert the text into graph documents
graph_documents = llm_transformer_filtered.convert_to_graph_documents(split_documents)

In [63]:
len(split_documents)

63

In [64]:
graph_documents

[GraphDocument(nodes=[Node(id="The Chronomancer's Emissary", type='Person', properties={}), Node(id='Chronomancer', type='Person', properties={})], relationships=[Relationship(source=Node(id="The Chronomancer's Emissary", type='Person', properties={}), target=Node(id='Chronomancer', type='Person', properties={}), type='WORKS_FOR', properties={})], source=Document(metadata={}, page_content='The Chronomancer’s Emissary\nPrologue')),
 GraphDocument(nodes=[Node(id='time itself', type='Concept', properties={}), Node(id='stars', type='Location', properties={}), Node(id='temporal energy', type='Energy', properties={}), Node(id='humanity', type='Organization', properties={}), Node(id='Chronomancers', type='Group', properties={})], relationships=[Relationship(source=Node(id='humanity', type='Organization', properties={}), target=Node(id='stars', type='Location', properties={}), type='CONQUERED', properties={}), Relationship(source=Node(id='humanity', type='Organization', properties={}), target=

In [65]:
ix = 0
print(f"Nodes:{graph_documents[ix].nodes}")
print(f"Relationships:{graph_documents[ix].relationships}")

Nodes:[Node(id="The Chronomancer's Emissary", type='Person', properties={}), Node(id='Chronomancer', type='Person', properties={})]
Relationships:[Relationship(source=Node(id="The Chronomancer's Emissary", type='Person', properties={}), target=Node(id='Chronomancer', type='Person', properties={}), type='WORKS_FOR', properties={})]


In [66]:
# Add the generated graph into Neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)


## Short Solution: 
#### Use GraphCypherQAChain to generate Cypher queries and return response from LLM

In [67]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# Who is Michael Scott?
MATCH (person:Person)-[:WON_AWARD]->(award:Award)
    WHERE person.id = 'Michael Scott'
RETURN person.id, award.id

# Who won the award?
MATCH (person:Person)-[:WON_AWARD]->(award:Award)
    WHERE person.id = 'Michael Scott'
RETURN person.id, award.id

# What kind of research does Michael conduct?
  MATCH (person:Person)-[:WORKED_ON]->(field:'Field of Study')
    WHERE person.id = 'Michael Scott'
  RETURN person.id, field.id

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [68]:
llm = ChatOllama(model="llama3")
chain = GraphCypherQAChain.from_llm(graph=graph, 
                                    llm=llm, 
                                    verbose=True, 
                                    allow_dangerous_requests=True, 
                                    # return_intermediate_steps=True, 
                                    # return_direct=True, 
                                    cypher_prompt=CYPHER_GENERATION_PROMPT,
                                    )
# response = chain.invoke({"query": "Who is Pierre Curie?"})

def prettyCypherChain(question: str) -> str:
    response = chain.run(question)
    print(textwrap.fill(response, 60))

# # To check the generated query manually
# cypher = """
# MATCH (person:Person)-[:WON_AWARD]->(award:Award) 
# WHERE person.id = 'Marie Curie' 
# RETURN person.id, award.id
# """
# graph.query(cypher)

In [69]:
prettyCypherChain("What is the story about?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (doc:Document)-[]->() 
RETURN doc.text
Full Context:
[{'doc.text': 'The Chronomancer’s Emissary\nPrologue'}, {'doc.text': 'The Chronomancer’s Emissary\nPrologue'}, {'doc.text': 'In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy,'}, {'doc.text': 'In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy,'}, {'doc.text': 'In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy,'}, {'doc.text': 'In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time 

## Longer Solution: 
#### Create a retriever from scratch

In [70]:
# Create embeddings for more complex search queries
embed = OllamaEmbeddings(model="nomic-embed-text")
vector_index = Neo4jVector.from_existing_graph(
    embedding=embed,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

# Define a model for the extracted entities from the text
class Entities(BaseModel):
    names: list[str] = Field(..., description="All entities from the text")

# Define a prompt to extract entities from the input query
prompt = ChatPromptTemplate.from_messages([ 
    ("system", "Extract organization and person entities from the text."),
    ("human", "Extract entities from: {question}")
])

# Initialize the Ollama model for entity extraction with LLM (using "llama3")
llm = OllamaFunctions(model="llama3", format="json", temperature=0.4)

# Combine the prompt and LLM to create an entity extraction chain
# The output is structured to match the "Entities" model
entity_chain = prompt | llm.with_structured_output(Entities, include_raw=True)

# Function to retrieve relationships of the extracted entities from Neo4j
def graph_retriever(question: str) -> str:
    # Use the entity extraction chain to get entities from the question
    response = entity_chain.invoke({"question": question})
    # Extract the list of entity names from the response
    entities = response['raw'].tool_calls[0]['args']['properties']['names']
    print("-"*30)
    print("Retreived Entities")
    print(entities)
    result = ""  # Initialize a variable to store the result

    # Iterate over each extracted entity
    for entity in entities:
        # Query Neo4j to get relationships for the given entity
        query_response = graph.query(
            """MATCH (p:Person {id: $entity})-[r]->(e)
            RETURN p.id AS source_id, type(r) AS relationship, e.id AS target_id
            LIMIT 50""",
            {"entity": entity}
        )
        # Format the query results and append to the result string
        result += "\n".join([f"{el['source_id']} - {el['relationship']} -> {el['target_id']}" for el in query_response])+ "\n"
    
    # Return the formatted results containing entity relationships
    return result

def full_retriever(question: str):
    # Retrieve graph data for the question using the graph_retriever function
    graph_data = graph_retriever(question)
    print("-"*30)
    print("Graph Data")
    print(graph_data)
    # Retrieve vector data by invoking the vector retriever with the question
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    # Retrieve vector-based data for multiple documents
    # vector_results = vector_retriever.invoke(question)
    #vector_data = "\n".join([f"#Document {i+1}: {el.page_content}" for i, el in enumerate(vector_results)])
    print("-"*30)
    print("Vector Data")
    print(vector_data)
    print("-"*30)
    # Combine the graph data and vector data into a formatted string
    return f"Graph data: {graph_data}\nVector data: {'#Document '.join(vector_data)}"

# Define a prompt template for generating a response based on context
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer:"""

# Create a prompt from the template, which takes the context and question as input
prompt = ChatPromptTemplate.from_template(template)

# Create a processing chain that:
# 1. Generates context using the full_retriever function
# 2. Passes through the question as-is using RunnablePassthrough
# 3. Applies the prompt template to generate the final question
# 4. Uses the LLM (language model) to generate the answer
# 5. Uses StrOutputParser to format the output as a string
chain = (
    {
        "context": lambda input: full_retriever(input),  # Generate context from the question
        "question": RunnablePassthrough(),  # Pass the question through without modification
    }
    | prompt  # Apply the prompt template
    | llm  # Use the language model to answer the question based on context
    | StrOutputParser()  # Parse the model's response as a string
)

In [74]:
# Test the chain with a question
response = chain.invoke(input="What is the story about?")
print("Final Answer")
print(textwrap.fill(response, 60))

------------------------------
Retreived Entities
{'type': 'array'}
------------------------------
Graph Data




Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

------------------------------
Vector Data
['\ntext: Kaden realized the stakes were higher than he had initially thought. This wasn’t just a single incident; it was part of a larger plan to rewrite human history.\nChapter 4: The Shadowed Ally', '\ntext: The Inhibitor also fed him real-time data about the surrounding area. Glancing down, he saw an alert flashing: Anomaly detected. Epicenter identified - East Village Laboratory, January 12, 2024.', '\ntext: Chapter 3: The Conspiracy', '\ntext: Chapter 1: The Ripple']
------------------------------
Final Answer
The story appears to be about a conspiracy or plot to
rewrite human history, with Kaden as the main character who
realizes the stakes are higher than initially thought. The
context suggests a mix of science fiction and mystery
elements.


>>> What is Isbank?

# Ollama Knowledge
IsBank (İştirak Bankası) was the first bank of Turkey. It was established 
in 1925 and merged with other banks to form İşbank (or simply İştirak 
Bankası), which was re-established on July 12, 1926, when it was 
privatized. The name "Is" is Turkish for "industry," and "bank."

# RAG w/o Graph
Isbank (officially known as Türkiye İş Bankası) is a commercial bank in Turkey.

# RAG w/Graph
İşbank, officially Türkiye İş Bankası, is a commercial bank in Turkey. Founded by the orders of Mustafa Kemal Atatürk in 1924, it is the first bank to go into operation in the Republic of Türkiye.


>>> What is the story about?

# RAG w/o Graph
Based on the given context, it appears that the story is
about something more significant than a minor mishap or
disturbance. The phrase "It's not just an object or a person
out of place" suggests that the story might be exploring a
larger issue or event that has consequences beyond a simple
mistake or error.

# RAG w/Graph
Approach #1: 
In the year 2589, humanity had conquered the stars and
fractured the fabric of time itself. Known as the
"Chronomancers," a select group of scientists harnessed
temporal energy, allowing them to traverse time and alter
its course. The story revolves around this concept.

Approach #2:
The story appears to be about a conspiracy or plot to
rewrite human history, with Kaden as the main character who
realizes the stakes are higher than initially thought. The
context suggests a mix of science fiction and mystery
elements.
